Dataset

In [1]:
X = ["\docume",
     "\date{\t",
     '''\centering
\begin{tabular}{|c|c|c|c|c|}
\hline X &
''',
     "\se",
     r'''\NewPart{Work experience}
\TimeEntry{07.2022 - 09.2022}{Junior engineer in power electronics}{at Trumpf Huettinger}{
\begin{compactitem}[-]
    \it''']

Tokenizer

In [2]:
import time
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablecode-completion-alpha-3b-4k")
# Add a padding token to the tokenizer

/home/laszer/.pyenv/versions/3.9.6/envs/stablecode_test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [4]:
len(tokenizer)

49153

In [5]:
X_encodings = tokenizer(X,  max_length=128,return_tensors="pt",truncation=True,padding=True)

In [6]:
from torch.utils.data import Dataset, DataLoader

class AutofillDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        label = self.labels[idx]

        return {
            "input_ids": input_id,
            "attention_mask": attention_mask,
            "labels": label
        }

# Encoded data
input_ids = X_encodings["input_ids"]
attention_mask = X_encodings["attention_mask"]
labels = X_encodings["input_ids"]

# Data creation
autofill_dataset = AutofillDataset(input_ids, attention_mask, labels)

# Dataloader creation
batch_size = 4
dataloader = DataLoader(autofill_dataset, batch_size=batch_size, shuffle=True)


In [7]:
batch_size = len(autofill_dataset)
print("Batch size:", batch_size)

Batch size: 5


FINE TUNING

In [8]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/stablecode-completion-alpha-3b-4k",
  trust_remote_code=True,
  torch_dtype="auto",
  load_in_8bit=True
)
# model.cuda()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
# model.enable_xformers_memory_efficient_attention()

In [10]:
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 49153. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(49153, 2560)

TRAIN

In [11]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    output_dir="./results",
    evaluation_strategy="steps",
    # eval_steps=500,
    save_total_limit=2,
    num_train_epochs=3,
    # fp16=True,
    # per_device_train_batch_size=8,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=autofill_dataset, #autofill_dataset
)

trainer.train()

100%|██████████| 15/15 [00:01<00:00,  7.98it/s]

{'train_runtime': 1.8788, 'train_samples_per_second': 7.984, 'train_steps_per_second': 7.984, 'train_loss': 6.154166666666667, 'epoch': 3.0}


TrainOutput(global_step=15, training_loss=6.154166666666667, metrics={'train_runtime': 1.8788, 'train_samples_per_second': 7.984, 'train_steps_per_second': 7.984, 'train_loss': 6.154166666666667, 'epoch': 3.0})